### Import

In [20]:
import requests
import datetime as dt
import json
import urllib.request
import urllib.parse

![nn](./documents/단기예보_조회서비스_상세기능명.png)

In [36]:
BASE_URL = ""

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
serviceKey = 'c2EtQlIYpThDFFRZclkoOmGaenwXMHJsGNg/sF49Oidra4IQo/s7moFPaGkJmTCzOZOPyfV/ByeQEnIjVIhvBA=='
now = dt.datetime.now()
year = now.year
month = now.month
day = now.day
date = "%d%02d%02d" %(year, month, day)

params ={'serviceKey' : serviceKey, 'pageNo' : '1', 'numOfRows' : '289', 'dataType' : 'JSON', 'base_date' : date, 'base_time' : '0500', 'nx' : '55', 'ny' : '127' }


response = requests.get(url, params=params)
result = json.loads(response.text)["response"]["body"]['items']['item']

In [37]:
for key in result:
    print(key)

{'baseDate': '20221011', 'baseTime': '0500', 'category': 'TMP', 'fcstDate': '20221011', 'fcstTime': '0600', 'fcstValue': '7', 'nx': 55, 'ny': 127}
{'baseDate': '20221011', 'baseTime': '0500', 'category': 'UUU', 'fcstDate': '20221011', 'fcstTime': '0600', 'fcstValue': '1.9', 'nx': 55, 'ny': 127}
{'baseDate': '20221011', 'baseTime': '0500', 'category': 'VVV', 'fcstDate': '20221011', 'fcstTime': '0600', 'fcstValue': '-1', 'nx': 55, 'ny': 127}
{'baseDate': '20221011', 'baseTime': '0500', 'category': 'VEC', 'fcstDate': '20221011', 'fcstTime': '0600', 'fcstValue': '298', 'nx': 55, 'ny': 127}
{'baseDate': '20221011', 'baseTime': '0500', 'category': 'WSD', 'fcstDate': '20221011', 'fcstTime': '0600', 'fcstValue': '2.1', 'nx': 55, 'ny': 127}
{'baseDate': '20221011', 'baseTime': '0500', 'category': 'SKY', 'fcstDate': '20221011', 'fcstTime': '0600', 'fcstValue': '1', 'nx': 55, 'ny': 127}
{'baseDate': '20221011', 'baseTime': '0500', 'category': 'PTY', 'fcstDate': '20221011', 'fcstTime': '0600', 'fc

In [80]:
import pandas as pd
df = pd.DataFrame(result)

In [81]:
from math import inf
columns = ['avg_temp','max_temp','min_temp','dif_temp','ave_humidity','precipitation']
# 평균기온 TMP, TMX, TMN
avg_temp = 0
# 최고기온
max_temp = -inf
# 최저기온
min_temp = inf
# 기온고저
dif_temp = 0
# 평균습도
ave_humidity = 0
# 강수량
precipitation = 0

#### 습도

In [82]:
REH = df[df['category']=='REH']['fcstValue'].apply(pd.to_numeric)
ave_humidity = REH.mean()
print(ave_humidity)

68.125


#### 강수량

In [90]:
PCP_df = df[df['category']=='PCP']
condition = PCP_df[PCP_df['fcstValue'].str.contains('강수없음')].index
PCP_df.drop(condition, inplace=True)
PCP = PCP_df['fcstValue'].apply(pd.to_numeric)
precipitation = PCP.sum()
print(precipitation)

0


C:\Users\JungHoimin\AppData\Local\Temp\ipykernel_13176\2632862832.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PCP_df.drop(condition, inplace=True)


#### 평균기온, 최고기온, 최저기온, 기온고저

In [97]:
temp_df = df[df['category'].isin(['TMP', 'TMX','TMN'])]['fcstValue'].apply(pd.to_numeric)
avg_temp = round(temp_df.mean(), 2)
max_temp = round(temp_df.max(), 2)
min_temp = round(temp_df.min(), 2)
dif_temp = round(max_temp - min_temp, 2)

print(avg_temp)
print(max_temp)
print(min_temp)
print(dif_temp)

11.24
18.0
7.0
11.0
